# geoDB Access

The geoDB is a service provided by the [Euro Data Cube project](https://eurodatacube.com) (EDC) as a payed service. It comes
with a Python client that provides hugh level acess to your data and a certain amount of space in a PostGreSQL database. For managing (as sharing is) 
your data you will need a management (read/write) account to your database which you can purchase at the [EDC market place](https://hub.eox.at).

You can access the service in two ways:

- By using the Jupyter Python notebook provided by EOX (configuartion free, ```geodb = GeoDBClient()```)
- By using you own Jupyter notebook or Python script by providing a client id and secret to the GeoDBClient (```geodb = GeoDBClient(client_id="myid", client_secret="mysecet")```)

The client ID and secret is also provided by EOX in the latter case. You will find them in your EOX hub account section. You can also provide the credentials
via system environment varibles (GEODB_AUTH_CLIENT_ID and GEODB_AUTH_CLIENT_SECRET). These variables can be supplied via a .env file. 

There are two different types of geoDB accounts: a read only and a management (read/write) access. The system will determine your access right through your 
authentication credentials.


## Sharing Data

In [1]:
from xcube_geodb.core.geodb import GeoDBClient

In [3]:
geodb = GeoDBClient()


In [9]:
geodb._auth_aud

'https://geodb.brockmann-consult.de'

In [3]:
import geopandas

# Have a look at fiona feature schema
collections = {
        "land_use":
        {
            "crs": 3794,
            "properties":
            {
                "RABA_PID": "float",
                "RABA_ID": "float",
                "D_OD": "date"
            }
        }
    }


geodb.create_collections(collections, clear=True)

gdf = geopandas.read_file('data/sample/land_use.shp')
geodb.insert_into_collection('land_use', gdf)

Processing rows from 0 to 9827


9827 rows inserted into land_use

### Publish a Collection to the World

In [4]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,demo,land_use,PUBLIC,SELECT
1,phi_week,gran_chaco,geodb_8c2d3fbe-f7a9-4492-8068-121e47e61a4f,"SELECT,SELECT"
2,phi_week,gran_chaco,geodb_a2b85af8-6b99-4fa2-acf6-e87d74e40431,"SELECT,SELECT"
3,phi_week,gran_chaco,geodb_d2c4722a-cc19-4ec1-b575-0cdb6876d4a7,"SELECT,SELECT"
4,phi_week,gran_chaco,PUBLIC,"SELECT,SELECT"


In [5]:
geodb.grant_access_to_collection("land_use", 'geodb_test5')

Access granted on land_use to geodb_test5

### Accessing the Collection as a different User

Let's access the collection as a different user (a test user in this case). You should now see a land_use collection.

In [6]:
geodb = GeoDBClient(client_id="93wdyTwOJebDozSCB4CubsWdFVKrSaXY", client_secret="zv0h3D3s90KAL640bw_MU4XMb6DfpJQloh877mH6Z1Nu4NrgPU2wSG3Ur_poUtqy")
geodb.whoami

'geodb_test5'

In [8]:
geodb.get_my_collections()

### Revoke access

Let's go back to the original user

In [39]:
geodb = GeoDBClient()
geodb.whoami


'geodb_admin'

In [40]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_admin,land_use,geodb_test5,SELECT


In [41]:
geodb.revoke_access_from_collection("land_use", 'geodb_test5')

Access revoked from geodb_admin on land_use

In [42]:
geodb.list_my_grants()

,Grants
0,No Grants


### Accessing the Collection as a different User

Let's list my collections again as a different user (a test user in this case). You should not see the above land use collection.

In [43]:
geodb = GeoDBClient(client_id="93wdyTwOJebDozSCB4CubsWdFVKrSaXY", client_secret="zv0h3D3s90KAL640bw_MU4XMb6DfpJQloh877mH6Z1Nu4NrgPU2wSG3Ur_poUtqy")
geodb.whoami

'geodb_test5'

In [44]:
geodb.get_my_collections()

,table_name
